[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/weaviate/recipes/blob/main/weaviate-features/model-providers/nvidia/similarity_search_nv_embed_v1.ipynb)

# Similarity Search with NVIDIA

This recipe will show you how to run semantic search queries with an embedding model on NVIDIA. 

## Requirements

1. Weaviate cluster
    1. You can create a 14-day free sandbox on [WCD](https://console.weaviate.cloud/)
    2. [Embedded Weaviate](https://weaviate.io/developers/weaviate/installation/embedded)
    3. [Local deployment](https://weaviate.io/developers/weaviate/installation/docker-compose#starter-docker-compose-file)
    4. [Other options](https://weaviate.io/developers/weaviate/installation)

2. NVIDIA NIM API key. Grab one [here](https://build.nvidia.com/models).
3. Weaviate client version `4.11.0` or newer
4. Weaviate database version `1.28.5`, `1.29.0`, or newer.

In [ ]:
!pip show weaviate-client

In [2]:
import weaviate, os
from weaviate.embedded import EmbeddedOptions
import weaviate.classes as wvc
import weaviate.classes.config as wc
import requests, json
import weaviate.classes.query as wq

## Connect to Weaviate

Only choose one option from the below.

**Weaviate Cloud Deployment**

In [6]:
WCD_URL = os.environ["WEAVIATE_URL"] # Replace with your Weaviate cluster URL
WCD_AUTH_KEY = os.environ["WEAVIATE_AUTH"] # Replace with your cluster auth key
NVIDIA_KEY = os.environ["NVIDIA_API_KEY"] # Replace with your NVIDIA key

# Weaviate Cloud Deployment
client = weaviate.connect_to_wcs(
    cluster_url=WCD_URL,
    auth_credentials=weaviate.auth.AuthApiKey(WCD_AUTH_KEY),
      headers={ "X-Nvidia-Api-Key": NVIDIA_KEY}
)

print(client.is_ready())

True


**Embedded Weaviate**

In [ ]:
# NVIDIA_KEY = os.environ["NVIDIA_API_KEY"] # Replace with your NVIDIA key

# client = weaviate.WeaviateClient(
#     embedded_options=EmbeddedOptions(
#         version="1.29.0",
#         additional_env_vars={
#             "ENABLE_MODULES": "text2vec-nvidia"
#         }),
#         additional_headers={
#             "X-Nvidia-Api-Key": NVIDIA_KEY
#         }
# )

# client.connect()

**Local Deployment**

In [ ]:
# NVIDIA_KEY = os.environ["NVIDIA_API_KEY"] # Replace with your NVIDIA key

# client = weaviate.connect_to_local(
#   headers={
#     "X-Nvidia-Api-Key": NVIDIA_KEY
#   }
# )
# print(client.is_ready())

## Create a collection
> Collection stores your data and vector embeddings.

In [9]:
# Note: in practice, you shouldn't rerun this cell, as it deletes your data
# in "JeopardyQuestion", and then you need to re-import it again.

# Delete the collection if it already exists
if (client.collections.exists("JeopardyQuestion")):
    client.collections.delete("JeopardyQuestion")

client.collections.create(
    name="JeopardyQuestion",

    vectorizer_config=wc.Configure.Vectorizer.text2vec_nvidia( # specify the vectorizer and model
        model="nvidia/nv-embed-v1", # optional, default is nvidia/nv-embed-v1
    ),

    properties=[ # defining properties (data schema) is optional
        wc.Property(name="Question", data_type=wc.DataType.TEXT), 
        wc.Property(name="Answer", data_type=wc.DataType.TEXT),
        wc.Property(name="Category", data_type=wc.DataType.TEXT, skip_vectorization=True), 
    ]
)

print("Successfully created collection: JeopardyQuestion.")

Successfully created collection: JeopardyQuestion.


## Import the Data

In [10]:
url = 'https://raw.githubusercontent.com/weaviate/weaviate-examples/main/jeopardy_small_dataset/jeopardy_tiny.json'
resp = requests.get(url)
data = json.loads(resp.text)

# Get a collection object for "JeopardyQuestion"
jeopardy = client.collections.get("JeopardyQuestion")

# Insert data objects
response = jeopardy.data.insert_many(data)

# Note, the `data` array contains 10 objects, which is great to call insert_many with.
# However, if you have a milion objects to insert, then you should spit them into smaller batches (i.e. 100-1000 per insert)

if (response.has_errors):
    print(response.errors)
else:
    print("Insert complete.")

Insert complete.


## Query Weaviate: Similarity Search (Text objects)

Similarity search options for text objects in **Weaviate**:

1. [near_text](https://weaviate.io/developers/weaviate/search/similarity#an-input-medium)

2. [near_object](https://weaviate.io/developers/weaviate/search/similarity#an-object)

3. [near_vector](https://weaviate.io/developers/weaviate/search/similarity#a-vector)

### `nearText` Example
Find a `JeopardyQuestion` about "animals in movies". Limit it to only 4 responses.

In [11]:
# note, you can reuse the collection object from the previous cell.
# Get a collection object for "JeopardyQuestion"
jeopardy = client.collections.get("JeopardyQuestion")

response = jeopardy.query.near_text(
    query="african beasts",
    limit=4
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", json.dumps(item.properties, indent=2), "\n")

ID: 9883d8b9-370a-498f-a490-8e9ca2244347
Data: {
  "answer": "Antelope",
  "question": "Weighing around a ton, the eland is the largest species of this animal in Africa",
  "category": "ANIMALS"
} 

ID: 761d964f-6e4a-45b4-a36c-622b0c75eba8
Data: {
  "answer": "Elephant",
  "question": "It's the only living mammal in the order Proboseidea",
  "category": "ANIMALS"
} 

ID: 26e392fb-dcb2-47c9-8467-f9d482d6267d
Data: {
  "answer": "species",
  "question": "2000 news: the Gunnison sage grouse isn't just another northern sage grouse, but a new one of this classification",
  "category": "SCIENCE"
} 

ID: c2f1a023-68fd-491b-86d5-fec9747e060c
Data: {
  "answer": "the nose or snout",
  "question": "The gavial looks very much like a crocodile except for this bodily feature",
  "category": "ANIMALS"
} 



**Return vector embeddings**

In [12]:
response = jeopardy.query.near_text(
    query="african beasts",
    include_vector=True,
    limit=4
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", json.dumps(item.properties, indent=2))
    print("Vector:", item.vector, "\n")

ID: 9883d8b9-370a-498f-a490-8e9ca2244347
Data: {
  "answer": "Antelope",
  "question": "Weighing around a ton, the eland is the largest species of this animal in Africa",
  "category": "ANIMALS"
}
Vector: {'default': [0.003520965576171875, 0.0117340087890625, -0.0005526542663574219, -0.008941650390625, 0.02301025390625, 0.03765869140625, -0.00849151611328125, -0.01800537109375, 0.009735107421875, 0.01158905029296875, -0.005863189697265625, 0.00785064697265625, 0.0029163360595703125, 0.0159759521484375, -0.02325439453125, -0.0030765533447265625, -0.0281829833984375, 0.007297515869140625, 0.00934600830078125, 0.00251007080078125, 0.006099700927734375, -0.01348114013671875, -0.0086212158203125, -0.01206207275390625, 0.01514434814453125, -0.0269927978515625, 0.0202484130859375, -0.006626129150390625, -0.002712249755859375, -0.017669677734375, -0.00337982177734375, 0.004055023193359375, 0.0144805908203125, 0.0031642913818359375, -0.00246429443359375, 0.0186309814453125, 0.007843017578125, 0

**Now, also request the `distance` for each returned item.**

In [13]:
response = jeopardy.query.near_text(
    query="african beasts",
    return_metadata=wq.MetadataQuery(distance=True),
    limit=4
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Distance:", item.metadata.distance)
    print("Data:", item.properties, "\n")

ID: 9883d8b9-370a-498f-a490-8e9ca2244347
Distance: 0.7335395216941833
Data: {'answer': 'Antelope', 'question': 'Weighing around a ton, the eland is the largest species of this animal in Africa', 'category': 'ANIMALS'} 

ID: 761d964f-6e4a-45b4-a36c-622b0c75eba8
Distance: 0.8691762685775757
Data: {'answer': 'Elephant', 'question': "It's the only living mammal in the order Proboseidea", 'category': 'ANIMALS'} 

ID: 26e392fb-dcb2-47c9-8467-f9d482d6267d
Distance: 0.9327269792556763
Data: {'answer': 'species', 'question': "2000 news: the Gunnison sage grouse isn't just another northern sage grouse, but a new one of this classification", 'category': 'SCIENCE'} 

ID: c2f1a023-68fd-491b-86d5-fec9747e060c
Distance: 0.9339777231216431
Data: {'answer': 'the nose or snout', 'question': 'The gavial looks very much like a crocodile except for this bodily feature', 'category': 'ANIMALS'} 



### `nearObject` Example

Search through the `JeopardyQuestion` class to find the top 4 objects closest to id `a1dd67f9-bfa7-45e1-b45e-26eb8c52e9a6`. (The id was taken from the query above)

In [15]:
response = jeopardy.query.near_object(
    near_object="9883d8b9-370a-498f-a490-8e9ca2244347", # replace with your id of interest
    limit=4
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", item.properties, "\n")

ID: 9883d8b9-370a-498f-a490-8e9ca2244347
Data: {'answer': 'Antelope', 'question': 'Weighing around a ton, the eland is the largest species of this animal in Africa', 'category': 'ANIMALS'} 

ID: 761d964f-6e4a-45b4-a36c-622b0c75eba8
Data: {'answer': 'Elephant', 'question': "It's the only living mammal in the order Proboseidea", 'category': 'ANIMALS'} 

ID: 26e392fb-dcb2-47c9-8467-f9d482d6267d
Data: {'answer': 'species', 'question': "2000 news: the Gunnison sage grouse isn't just another northern sage grouse, but a new one of this classification", 'category': 'SCIENCE'} 

ID: ee1a49a7-e0ab-4d40-85c1-fc26cbf255e6
Data: {'answer': 'the diamondback rattler', 'question': 'Heaviest of all poisonous snakes is this North American rattlesnake', 'category': 'ANIMALS'} 



### `nearVector` Example
Search through the `JeopardyQuestion` class to find the top 2 objects closest to the query vector `[-0.0125526935, -0.021168863, ... ]`

In [ ]:
response = jeopardy.query.near_vector(
    near_vector=[-0.0125526935, -0.021168863, ... ], # your vector object goes here
    limit=4
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", item.properties, "\n")